In [1]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [3]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [6]:
array_aePerulangan = []
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter

    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 10 #70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(lower_bound=5, upper_bound=7.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(lower_bound=7.5, upper_bound=12.49, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(lower_bound=12.5, upper_bound=15, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(lower_bound, upper_bound)
            return particles_position[i]
    
    swarm_size = 5 #70 
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    array_aeParticle = []
    ae_minimum = 5000
    
    for i in range(iteration_max):
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        for j in range(swarm_size):    
                
            xSimple = randomSimpleUCW()
            xAverage = randomAverageUCW()
            xComplex = randomComplexUCW()
            case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
            estimated_effort = case * 20
        
            #Selisih estimasi effort
            ae = abs(estimated_effort - real_effort)
            
            #Menyimpan ae minimum dan letaknya
            if ae_minimum > ae:
                ae_minimum = ae
                letak_i = i
                letak_j = j
                xSimpleMin = xSimple
                xAverageMin = xAverage
                xComplexMin = xComplex
            
            #Memasukkan selisih
            #array_aeParticle.append(ae)
            #aeParticle_min = np.min(array_aeParticle)
            #index_of_minimum = np.where(array_aeParticle == aeParticle_min)
        
            #menjumlah 10 angka selisih
            temp = np.array(temp+ae)
         
            #print('Partikel ke', j,':',estimated_effort)
            #print('ucp   ', case)
            print('ae ', j, ae)
            #print('S     ', xSimple)
            #print('A     ', xAverage)
            #print('C     ', xComplex)
            #print()
            
            #print('estimated effort', estimated_effort)
            
            if j == 0:
                pbest_simple = xSimple
                pbest_average = xAverage
                pbest_complex = xComplex
                
                vSimple = random.uniform(0, 1)
                vAverage = random.uniform(0, 1)
                vComplex = random.uniform(0, 1)
                
                xSimple = pbest_simple
                xAveragepbest_average = pbest_average
                xComplex = pbest_complex
        
        #print('ae terkecil : ',np.min(aeParticle_min))
        #print('letak ae : ',index_of_minimum)
        #print('pada perulangan ke :', aeParticle_min.index(min(aeParticle_min)))
        #mae = temp/swarm_size
        #print('MAE :', mae)
        print()
        #array_aePerulangan.append(aeParticle_min)
    print('ae_minimum', ae_minimum)
    print('i', letak_i)
    print('j', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)
    #print('ae terkecil keseluruhan : ',np.min(array_aePerulangan))
    #print('letak ae : ',index_of_minimum)


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.25278693589056533
r2 : 0.6154236295135417

ae  0 3314.5442857490534
ae  1 3092.158828149738
ae  2 3273.0788988789027
ae  3 3338.7121906698612
ae  4 3412.9321075097014

Perulangan  1
inertia : 0.85
r1 : 0.7597115602492343
r2 : 0.6691993577791893

ae  0 3264.1906394438683
ae  1 2863.06112050062
ae  2 3253.077928321536
ae  3 3440.69684009121
ae  4 3459.2106345160964

Perulangan  2
inertia : 0.8
r1 : 0.1385379203389313
r2 : 0.9240760985426019

ae  0 3029.8479039553067
ae  1 2877.9213471543017
ae  2 3238.4290065006207
ae  3 3206.0846145921178
ae  4 3447.0113264031133

Perulangan  3
inertia : 0.75
r1 : 0.6433912523875237
r2 : 0.506770997969372

ae  0 2855.4825054844787
ae  1 2810.216519029914
ae  2 3415.9746251548513
ae  3 2938.175579881331
ae  4 3301.168392465509

Perulangan  4
inertia : 0.7
r1 : 0.3363875483603067
r2 : 0.4345693173334131

ae  0 3415.0699709711853
ae  1 3257.5059625614467
ae  2 3248.5393803857996
ae  3 2999.030830395219


ae  3 1330.0109546977974
ae  4 2273.0714323420707

Perulangan  1
inertia : 0.85
r1 : 0.8027867763599121
r2 : 0.980167087971201

ae  0 2217.713543479387
ae  1 2312.7129179550375
ae  2 1463.4933048915555
ae  3 2246.2799186309003
ae  4 1785.4159505836442

Perulangan  2
inertia : 0.8
r1 : 0.6388160727168735
r2 : 0.7234466260064738

ae  0 1412.215552228825
ae  1 2325.401539641036
ae  2 2592.660537275249
ae  3 1229.2193826737112
ae  4 2261.020432961742

Perulangan  3
inertia : 0.75
r1 : 0.9983037997944484
r2 : 0.49108190506758476

ae  0 1958.476016463489
ae  1 1618.8549778444449
ae  2 1971.3744231511419
ae  3 2426.538768835636
ae  4 2041.3855132201534

Perulangan  4
inertia : 0.7
r1 : 0.04432420584978958
r2 : 0.23605973990218176

ae  0 2177.5335476842774
ae  1 1986.8397342814987
ae  2 2176.13485755093
ae  3 2227.8759423883257
ae  4 2081.322454461958

Perulangan  5
inertia : 0.65
r1 : 0.4843219149318889
r2 : 0.38963117795641367

ae  0 1385.584030595619
ae  1 2591.5027966388925
ae  2 2130.8422

ae  3 2429.5879432820584
ae  4 2899.917076505042

Perulangan  3
inertia : 0.75
r1 : 0.18439983149043515
r2 : 0.21997540433310014

ae  0 2312.460013487881
ae  1 2437.0972220436042
ae  2 2199.0353342923863
ae  3 2706.4455288673125
ae  4 2697.4522897494453

Perulangan  4
inertia : 0.7
r1 : 0.019324395640130332
r2 : 0.3438214493125076

ae  0 2582.0719101877867
ae  1 2321.7911607146943
ae  2 2561.2509721112265
ae  3 2689.5363366207666
ae  4 2484.6270438760766

Perulangan  5
inertia : 0.65
r1 : 0.3999845435720063
r2 : 0.2690424790391608

ae  0 2557.4740675703106
ae  1 2606.4853785129353
ae  2 2522.577516360052
ae  3 2608.7169669000978
ae  4 2505.3346650940985

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.01984114928532521
r2 : 0.1734125276502182

ae  0 2330.3236126660095
ae  1 2125.242839332691
ae  2 2233.168957940159
ae  3 2219.858654999606
ae  4 2311.6097476871482

Perulangan  7
inertia : 0.55
r1 : 0.34462821057020177
r2 : 0.8898054274694114

ae  0 2580.884014228822
ae  1 2742.5443746

ae  4 2311.4155763994377

Perulangan  9
inertia : 0.45
r1 : 0.13379691433219532
r2 : 0.9274103086946147

ae  0 2032.1751562633572
ae  1 2839.971567073162
ae  2 1771.903029137704
ae  3 2685.904588555195
ae  4 2296.4343623132427

ae_minimum 1771.903029137704
i 9
j 2
S Min 6.57704401841861
A Min 12.445991037621933
C Min 14.819665301482129

____ DATA  18 ____
Perulangan  0
inertia : 0.9
r1 : 0.10494502147779816
r2 : 0.7621892363376467

ae  0 1078.7498745252724
ae  1 1903.2855572928247
ae  2 1514.4321667420645
ae  3 1626.0297837392673
ae  4 1240.2721766117293

Perulangan  1
inertia : 0.85
r1 : 0.7073411009341305
r2 : 0.23992815718850835

ae  0 1472.9308988770426
ae  1 1321.5818607944393
ae  2 1487.1018578043459
ae  3 1864.5500017145023
ae  4 1832.311038458065

Perulangan  2
inertia : 0.8
r1 : 0.23894689623834686
r2 : 0.6073135284006295

ae  0 1048.7306054949831
ae  1 1559.784451134472
ae  2 1800.0734670514212
ae  3 1715.3450484407394
ae  4 1671.8524293381415

Perulangan  3
inertia : 0.75
r1

inertia : 0.45
r1 : 0.16217088581735195
r2 : 0.8615680901797015

ae  0 2225.642643455287
ae  1 2893.034489269041
ae  2 2136.3556043733724
ae  3 2415.1401663103898
ae  4 2627.983140728541

ae_minimum 1832.2879354006936
i 1
j 0
S Min 5.961142056297437
A Min 12.095165108279982
C Min 14.74919059462953

____ DATA  25 ____
Perulangan  0
inertia : 0.9
r1 : 0.8666551527871222
r2 : 0.6698292216559484

ae  0 2893.581324159668
ae  1 2091.2547957564693
ae  2 1989.7640060711656
ae  3 1742.8643347942516
ae  4 2425.8547513357244

Perulangan  1
inertia : 0.85
r1 : 0.525130031580845
r2 : 0.1498950377598106

ae  0 2595.840237174585
ae  1 2037.4846700184598
ae  2 2603.363954481313
ae  3 2416.7033923537147
ae  4 2248.8749354862466

Perulangan  2
inertia : 0.8
r1 : 0.95636860238686
r2 : 0.8021656608300639

ae  0 2435.97080562871
ae  1 2183.387645642154
ae  2 2111.370474183721
ae  3 1950.0228515341387
ae  4 2211.7387902666933

Perulangan  3
inertia : 0.75
r1 : 0.13632407119680678
r2 : 0.2905963362534306

ae

...

ae  1 2267.7757510367974
ae  2 3299.8090202635285
ae  3 2887.1310291324407
ae  4 2300.7774038240823

Perulangan  5
inertia : 0.65
r1 : 0.4511931550186177
r2 : 0.6200441025492461

ae  0 2812.9266358593304
ae  1 1925.5709680627006
ae  2 2906.107688230608
ae  3 2320.5181677315522
ae  4 1561.6186899591976

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.8245628422143715
r2 : 0.17121460305079605

ae  0 2149.0965443690693
ae  1 1409.1963955257352
ae  2 1389.9370771540753
ae  3 2788.6123772407536
ae  4 3106.0079632180114

Perulangan  7
inertia : 0.55
r1 : 0.29854742263851
r2 : 0.850622496442784

ae  0 1161.5492771208883
ae  1 2160.7709960845095
ae  2 1615.6683714761357
ae  3 1640.3496917018892
ae  4 1808.627926939016

Perulangan  8
inertia : 0.5
r1 : 0.7589765208911827
r2 : 0.9370298614202085

ae  0 3321.458558374554
ae  1 2385.259551420204
ae  2 1491.1844398784015
ae  3 3184.6939858081587
ae  4 2999.7775501761353

Perulangan  9
inertia : 0.45
r1 : 0.41682121392399074
r2 : 0.403344081213839

Perulangan  4
inertia : 0.7
r1 : 0.43660196907648663
r2 : 0.1574864972701634

ae  0 1705.23741399947
ae  1 1040.9474188560498
ae  2 813.5198078186295
ae  3 1654.9171347616284
ae  4 713.8321552358266

Perulangan  5
inertia : 0.65
r1 : 0.051636901809705704
r2 : 0.06931440833129865

ae  0 1015.6979091806115
ae  1 1646.5685700454706
ae  2 2030.3520666558707
ae  3 1012.3465168747389
ae  4 867.5092395646925

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.4407772887562915
r2 : 0.814923564808108

ae  0 592.17359405561
ae  1 1138.844608469628
ae  2 1128.9895557191985
ae  3 1460.179035442311
ae  4 701.2339771536008

Perulangan  7
inertia : 0.55
r1 : 0.6216138996761106
r2 : 0.09415494374963951

ae  0 1768.6486049720934
ae  1 726.3353686687069
ae  2 1524.5132920117849
ae  3 1989.4638061239393
ae  4 1523.4853100228502

Perulangan  8
inertia : 0.5
r1 : 0.15883341123624084
r2 : 0.1849838521576035

ae  0 749.5868670566069
ae  1 1120.1085902446048
ae  2 1459.6350044692808
ae  3 1191.030960720017
ae 


ae  0 360.83253559898367
ae  1 696.0276426650325
ae  2 771.9439840710829
ae  3 575.6870258052368
ae  4 1054.1899299027218

Perulangan  9
inertia : 0.45
r1 : 0.9241903732088675
r2 : 0.8438491037517333

ae  0 1056.5219812603327
ae  1 768.4931546913595
ae  2 1528.6766098534572
ae  3 1728.477388707016
ae  4 625.6418289468456

ae_minimum 220.55888682409477
i 5
j 4
S Min 5.8710693685755375
A Min 12.467577777986584
C Min 14.950418624847277

____ DATA  58 ____
Perulangan  0
inertia : 0.9
r1 : 0.9472212280609436
r2 : 0.7878906750932514

ae  0 2068.672321243872
ae  1 2020.134789217259
ae  2 1578.091423341919
ae  3 2190.0821643878426
ae  4 1760.4959098925792

Perulangan  1
inertia : 0.85
r1 : 0.22845461086885743
r2 : 0.7937579952493083

ae  0 1096.8892926482795
ae  1 516.4612429877634
ae  2 1847.2360943761578
ae  3 982.3137329727215
ae  4 1131.039304017133

Perulangan  2
inertia : 0.8
r1 : 0.24213005446350222
r2 : 0.5840331337912485

ae  0 1423.740276608727
ae  1 2027.900464046742
ae  2 1255.081

ae  0 1468.2994636311105
ae  1 1270.6651784000842
ae  2 1230.550350780306
ae  3 1117.7796337258278
ae  4 996.0095453887616

Perulangan  4
inertia : 0.7
r1 : 0.8922470058163449
r2 : 0.7302425822194788

ae  0 1049.4993475008223
ae  1 1496.2684023473857
ae  2 1129.0245825068132
ae  3 1388.5561590689686
ae  4 1263.3199861608718

Perulangan  5
inertia : 0.65
r1 : 0.8520445692918723
r2 : 0.3860444249548408

ae  0 1060.5822260569103
ae  1 1027.4930596387849
ae  2 1514.7253052487113
ae  3 953.3740519295579
ae  4 973.5256888012864

Perulangan  6
inertia : 0.6000000000000001
r1 : 0.9103014326213709
r2 : 0.5665103318413797

ae  0 1194.7582576692248
ae  1 1100.1294517801643
ae  2 902.9362536753351
ae  3 917.9098218492227
ae  4 1327.494923684706

Perulangan  7
inertia : 0.55
r1 : 0.9156950008663562
r2 : 0.8401667291230229

ae  0 1409.910856542725
ae  1 970.2767653247174
ae  2 946.985930712498
ae  3 864.1759060024287
ae  4 1584.7251934954056

Perulangan  8
inertia : 0.5
r1 : 0.014928486042634637
r2 

ae  3 3002.4977744586486
ae  4 3509.692854925446

Perulangan  8
inertia : 0.5
r1 : 0.3089381026146374
r2 : 0.0914092566238569

ae  0 1537.8083153696334
ae  1 3467.5146135830364
ae  2 2775.052868419767
ae  3 1632.6339235952964
ae  4 2188.628836096652

Perulangan  9
inertia : 0.45
r1 : 0.8094188390159046
r2 : 0.3443614269684818

ae  0 2894.8137538720675
ae  1 3494.4492839143204
ae  2 2362.33166860662
ae  3 2362.874871946611
ae  4 2452.513050821668

ae_minimum 1412.255161489953
i 3
j 0
S Min 5.8795527624975
A Min 7.778847223470683
C Min 12.98074808958607

____ DATA  70 ____
Perulangan  0
inertia : 0.9
r1 : 0.06447630623637313
r2 : 0.7365759768957512

ae  0 550.6487124842806
ae  1 1818.0555782069123
ae  2 1092.5589484608472
ae  3 1615.0948911381947
ae  4 1183.637841303097

Perulangan  1
inertia : 0.85
r1 : 0.2564035202457735
r2 : 0.5785426751317982

ae  0 776.854267534678
ae  1 701.576928772176
ae  2 743.7833581430732
ae  3 713.6765801901838
ae  4 918.9686880054205

Perulangan  2
inertia :